In [2]:
!pip install mediapipe opencv-python

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [6]:
# 1. MAKE DETECTIONS

cap = cv2.VideoCapture(0)

## SETUP MEDIAPIPE instance

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
            ## recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
        image.flags.writeable = False
        
            # Make detections
        results = pose.process(image)

            # recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # extract landmark
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
                
            #render image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color = (245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color = (245,66,230), thickness=2, circle_radius=2)
                                )
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break   
cap.release()
cv2.destroyAllWindows()

D:\jupyter\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[x: 0.573015332
y: 0.729772449
z: -2.89979744
visibility: 0.991139352
, x: 0.618570924
y: 0.642504692
z: -2.86809182
visibility: 0.988408148
, x: 0.643945456
y: 0.639309168
z: -2.86823511
visibility: 0.987620115
, x: 0.66555655
y: 0.637381792
z: -2.86893106
visibility: 0.986232638
, x: 0.539208114
y: 0.646566927
z: -2.86403942
visibility: 0.98851192
, x: 0.517023802
y: 0.645024419
z: -2.86469698
visibility: 0.987587094
, x: 0.500830948
y: 0.644265175
z: -2.86513638
visibility: 0.988213897
, x: 0.709559083
y: 0.644641161
z: -2.27387214
visibility: 0.988811851
, x: 0.492460191
y: 0.648172617
z: -2.24742484
visibility: 0.991166294
, x: 0.623784542
y: 0.807782769
z: -2.6370647
visibility: 0.988027811
, x: 0.542687178
y: 0.808560967
z: -2.63261986
visibility: 0.988220692
, x: 0.885501683
y: 0.918198287
z: -1.55923498
visibility: 0.95766151
, x: 0.337297946
y: 0.951496482
z: -1.47999763
visibility: 0.961092412
, x: 0.983910382
y: 1.27540195
z: -1.06821942
visibility: 0.135701135
, x: 0.22412

KeyboardInterrupt: 

In [8]:
len(landmarks)

33

In [10]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [12]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

x: 0.384405673
y: 1.24632335
z: -1.08135688
visibility: 0.00767986104

In [14]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

0.9675140976905823

In [16]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.341651857
y: 0.953940392
z: -1.7133162
visibility: 0.941085339

In [18]:
## 3 ACUTE ANGLES

def calculate_angle(a,b,c):
    a = np.array(a) # first
    b = np.array(b) # mid
    c = np.array(c) # end

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360.0 - angle
    return angle

In [20]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y] 
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y] 
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] 

In [22]:
shoulder, elbow, wrist

([0.8816772699356079, 0.8828696012496948],
 [0.9545331001281738, 1.244297742843628],
 [0.9286531209945679, 1.4949520826339722])

In [24]:
calculate_angle(shoulder, elbow, wrist)

162.70829632674972

In [26]:
tuple(np.multiply(elbow,[640,480]).astype(int))

(610, 597)

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
